# Module 5 Exercises: Deployment

**Objective**: Master Docker optimization and Kubernetes debugging.

---

## 🛠️ Setup

```bash
pip install fastapi uvicorn
```

## 🔎 Part 1: The Docker Audit

**Scenario**: The following Dockerfile takes 10 minutes to build every time you change a single line of code in `app.py`. **Why?**

```dockerfile
FROM python:3.8
WORKDIR /app
COPY . .           <-- PROBLEM HERE
RUN pip install -r requirements.txt
CMD ["python", "app.py"]
```

<details>
<summary><b>🔻 Click to Reveal Diagnosis</b></summary>
<br>
<b>Issue:</b> `COPY . .` copies all source code. If `app.py` changes, the Docker cache for this layer is invalidated. 
Since the next line is `RUN pip install ...`, Docker forces a full re-install of pandas/numpy/torch every single time.
<br>
<b>Fix:</b> 
1. `COPY requirements.txt .`
2. `RUN pip install ...` (Now cached unless requirements change)
3. `COPY . .` (Only this fast layer re-runs)
</details>

## 🩺 Part 2: Pydantic Validation Lab

**Task**: Create a User model that rejects the `invalid_user` dictionary below.
Constraints:
- Age must be >= 18
- Email must contain '@'

In [ ]:
from pydantic import BaseModel, Field, validator

class User(BaseModel):
    username: str
    age: int = Field(..., ge=18)
    email: str
    
    @validator("email")
    def validate_email(cls, v):
        if "@" not in v:
            raise ValueError("Invalid email format")
        return v

invalid_user = {"username": "kid", "age": 15, "email": "not_an_email"}

try:
    User(**invalid_user)
except Exception as e:
    print(f"✅ Caught Error: {e}")

## 🚑 Part 3: K8s Debugging Flowchart

**Scenario**: Your pod is in `CrashLoopBackOff`.
Arrange the following commands in the order you would run them.

A) `kubectl logs pod-name`
B) `kubectl delete pod pod-name`
C) `kubectl describe pod pod-name`

<details>
<summary><b>🔻 Click for Answer</b></summary>
<br>
<b>Order: C -> A -> B</b>.

1. <b>Describe (C)</b>: Tells you *why* it crashed (OOMKilled? ImagePullBackOff?).
2. <b>Logs (A)</b>: Tells you if the *application* threw an error (Python Exception).
3. <b>Delete (B)</b>: The last resort. K8s will restart it, but if you don't fix the bug, it will just crash again.
</details>